In [3]:
import pandas as pd
import geopandas as gpd
import json 
from shapely.geometry import shape, Point
import shapely

import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import folium

from vincenty import vincenty

import requests
import urllib.parse
import re

from sklearn import preprocessing


# Option d'affchage
pd.set_option('display.max_columns', None)

In [ ]:
# Si la base de données n'est pas déjà téléchargée
# df0 = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/3004168d-bec4-44d9-a781-ef16f41856a2', sep = '|')

# Sinon
df0 = pd.read_csv('data/valeursfoncieres-2019.txt', sep = '|')

# I. Nettoyage de la base de données

In [ ]:
df1 = df0.copy()

df1.sample(5)

## 1. Séléction des variables intéressantes

In [ ]:
df1.columns

In [ ]:
df = df1[[
        'Date mutation', 'Nature mutation', 'Valeur fonciere',
        'No voie', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
        'Commune', 'Code departement', 'Code commune', 'Type local',
        'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain',
        'Section', 'No plan'
        ]]

df.sample(5)

### Premier tri pour garder les biens qui nous intéressent dans le cadre que l'on s'est donné

In [ ]:
df = df[df['Nature mutation'] == 'Vente']
df = df[df['Code departement'] == 75]
df = df[df['Type local'] == 'Appartement']
df = df[pd.isna(df['Surface terrain'])] # Vu qu'on va essayer de prédir un prix au m2, la surface est une donnée indispensable

### Changement du nom des variables

In [ ]:
df = df[[
        'Date mutation', 'Valeur fonciere', 'No voie', 
        'Type de voie', 'Voie', 'Code postal', 'Surface reelle bati', 
        'Nombre pieces principales',
        'Code departement', 'Code commune', 'Code voie',
        'Type local', 'Section', 'No plan'
        ]]

df = df.rename(columns = {
                            'Date mutation': 'Date', 
                            'Valeur fonciere': 'Valeur',
                            'Type de voie': 'TypeVoie',
                            'No voie': 'Numero',
                            'Code postal': 'CodePostal',
                            'Surface reelle bati': 'Surface',
                            'Nombre pieces principales': 'NbPieces',
                            'Surface terrain': 'SurfaceTerrain',
                            'Code departement': 'Code_departement', 
                            'Code commune': 'Code_commune', 
                            'Code voie': 'Code_voie',
                            'Type local': 'TypeLocal',
                            'No plan': 'No_plan'
                         }
              )

## 2. Gestion des variables

In [ ]:
df.dtypes

On remarque que la variable "Valeur" n'est pas de type float ou int, il va falloir y remédier !

### Valeur

In [ ]:
# Les biens dont la valeur n'est pas renseignée ne nous intéressent pas
df = df.dropna(subset = ['Valeur'])

# Changement des virgules en point pour pouvoir convertir les types str en types float
df['Valeur'] = df.apply(lambda row : str(row.Valeur).replace(',', '.'), axis = 1)
df['Valeur'] = pd.to_numeric(df['Valeur'])

# On divise par 1000 les prix pour plus de lisibilité
df['Valeur'] = df['Valeur']/1000

# On se concentre sur une tranche de prix "raisonnable"
df = df[df['Valeur'] > 60]
df = df[df['Valeur'] < 15000]

# Pour déterminer ces bornes, nous sommes allé sur des sites d'immobiliers à Paris pour trouver les valeurs extrêmes

### Surface

Un logement mis en location doit respecter une surface minimum. Il s'agit d'un des critères de décence du logement.

Le logement doit comporter au moins une pièce principale présentant :

une surface habitable de 9 m² et une hauteur sous plafond minimale de 2,20 mètres,
ou un volume habitable de 20 m³.

In [ ]:
df = df[df['Surface'] >= 9]

### Création de la variable du prix par metre carré (prixm2)

In [ ]:
df['prixm2'] = df['Valeur'] / df['Surface']

### Date

In [ ]:
display(df.sort_values('Date')[['Date']].head())
df[pd.isna(df['Date'])].shape

# Il ne semble pas y avoir de valeurs manquantes

### Elements d'adresse (Numero, TypeVoie,  Voie, CodePostal)

In [ ]:
# Numero
print(df[pd.isna(df['Numero'])].shape)

# Tous les appartments ont un numéro indiqué. On est satisfait car on souhaite une localisation précise des appartements

In [ ]:
# TypeVoie

print(df[pd.isna(df['TypeVoie'])].shape)

# 15 valeurs manquantes, on regarde à quoi elles correspondent

display(df[pd.isna(df['TypeVoie'])])

# Cela correspond à des adresses où le type de voie est spécial (Villa, Pont, Autoroute, Rond point) 
# et est contenu dans la variable Voie
# On laisse comme ca

In [ ]:
# Voie
display(df.sort_values('Voie')[['Voie']].head())
print(df[pd.isna(df['Voie'])].shape)

# Il ne semble pas y avoir de valeurs manquantes

In [ ]:
# CodePostal

print(df[pd.isna(df['CodePostal'])].shape)

# 1 valeur manquante, on regarde à quoi elle correspond

display(df[pd.isna(df['CodePostal'])])

# La rue de l'Abbé Groult se situe dans le 15ème, on remplit à la main

df.at[2503844, 'CodePostal'] = float(75015)

display(df.loc[[2503844]])

### NbPieces (nombre de pièces)

In [ ]:
print(df[pd.isna(df['NbPieces'])].shape)

display(df.sort_values('NbPieces')[['NbPieces']].head())

# Pas de valeurs manquantes mais des appartements à 0 pièces...

display(df[df['NbPieces'] == 0].head())

# Les entrées n'ont pas l'air des anomalies, la valeur doit être manquante, on laisse tel quel en gardant à l'esprit cette observation

#### Création de la variable 'id' qui permettra d'identifier l'adresse précise dans une autre bdd

In [ ]:
def code_numero(numero):
    """Créé le code utilisé pour identifier le numero d'une rue au sein du code 'id' 

    Argument :
    numero : float
        numero de la rue tel que présent dans la base de données

    Return :
    code : str
        Le code correspondant (format 00007 pour le numéro 7 d'une rue, par exemple)
    """

    code = str(int(numero))
    code = '0' * (5 - len(code)) + code

    return code

df['id'] = df['Code_departement'].astype(str) + df['Code_commune'].astype(str) + '_' + df['Code_voie'].astype(str) + '_' + df['Numero'].apply(code_numero)

df = df.drop(columns = ['Code_commune', 'Code_departement', 'Code_voie'])

# Vérification

df.head(5)

In [ ]:
# Petite sauvegadre de la base à ce stade

dfv0 = df.copy()

In [ ]:
df = dfv0.copy()

## 3. Choix de la variable à prédir

In [ ]:
# On peut tracer le prix en fonction de la surface

df[df['Valeur'] < 2000].sample(2000).plot(x = 'Surface', y = 'Valeur', kind = 'scatter', alpha = .1)
# On a tracé le graphe pour les appartements ayant un prix inférieur à 2 millions

On voit une relation linéaire apparaitre, justifiée par un coefficient de corrélation significatif, de 0.58 

In [ ]:
df['Valeur'].corr(df['Surface'])

#### On fait donc l'hypothèse qu'il y a une corélation linéaire entre le prix d'un appartement et sa surface en m2.
#### On se donnera donc pour objectif de prédir le prix au m2 d'un appartement en fonction de différents paramètres.

## 4. Gestion des anomalies

La base de données n'est pas parfaite, certaines entrées sont erronées.
La premier nettoyage que nous allons faire, qui est aussi le plus grossier, est de se restreindre à des prix au m2 raisonnables.
#### Le site meilleursagents.com recense les prix au m2 de nombreux appartements à Paris. 
#### Les prix les plus bas observé sont environ de 4 700 €/m2
#### Les prix les plus hauts observés sont environ de 32 000 €/m2

#### On choisit donc de considérer les biens dont le prix au m2 est compris entre 4 230 € et 35 200 (marge de 10%)

In [ ]:
df = df[df['prixm2'] > 4.23]
df = df[df['prixm2'] < 35.2]

df.plot(x = 'Surface', y = 'Valeur', kind = 'scatter', alpha = .1)

In [ ]:
df['Valeur'].corr(df['Surface']) # Nouveau coefficient de corrélation : 0.88

# Couplage : date de construction du bâtis

Pour obtenir la date de construction des immeubles, on utilise deux nouvelles bases :  
    -df_dates qui donne la date de construction à partir d'un identificateur id2  
    -df_join qui associe l'identificateur id2 à une adresse
    
   df_join va donc nous permettre de faire la jointure entre la base d'origine et la base df_dates pour obtenir la date deconstruction des immeubles.

In [ ]:
# Importation des bases
df_dates = pd.read_csv('data/date_construction.csv', sep = ',')
df_join = pd.read_csv('data/PARCELLE_CADASTRALE.csv', sep = ',')

In [ ]:
# On garde que les variables qui nous intéressent
df_join = df_join[['n_sq_pc', 'c_sec', 'n_pc']]
df_dates = df_dates[['n_sq_pc', 'c_perconst']]

In [ ]:
# On regroupe les deux bases grâce à l'identificateur
df_dates = df_dates.merge(df_join, left_on = 'n_sq_pc', right_on = 'n_sq_pc')
df_dates = df_dates.drop(columns = ['n_sq_pc'])

In [ ]:
# On crée l'identificateur dans la base d'origine et le fait correspondre avec celui de la base des dates
df['id2'] = df.apply(lambda row: str(row.Section) + str(row.No_plan), axis = 1)
df_dates['id2'] = df_dates.apply(lambda row: str(row.c_sec) + str(row.n_pc), axis = 1)
df_dates = df_dates.drop_duplicates(subset = ['id2'], keep = 'first')

display(df_dates.head(10))
display(df.head(10))


# On joint les deux bases
df = pd.merge(df, df_dates, on = 'id2')

In [ ]:
# Et on nettoie le tout
df = df.drop(columns = ['id2', 'Section', 'No_plan'])
df = df.rename(columns = {'c_perconst': 'periode_construction'})

In [ ]:
px.histogram(df, x = 'periode_construction')

In [ ]:
# La catégorie 4 n'existe pas, on recode pour que ca soit plus compréhensif
def ajustement(valeur):
    if valeur in [i for i in range(5, 13)]:
        return valeur - 1
    elif valeur == 99:
        return 0
    else:
        return valeur

df['periode_construction'] = df.apply(lambda row: ajustement(row.periode_construction), axis = 1)


In [ ]:
px.histogram(df, x = 'periode_construction')

In [ ]:
df.shape

On obtient ici plusieurs tranches de périodes de constructions :  
0   Données manquantes  
1	Avant 1800	
2	1801-1850	
3	1851-1914	
4	1915-1939	
5	1940-1967	
6	1968-1975	
7	1976-1981	
8	1982-1989	
9	1990-1999	
10	2000-2007	
11	2008 et plus

# A partir de maintenant on travail avec une petite portion de la base pour éviter les calculs trop longs.
# La base a été traitée en entier par nos soins en amont, puis sauvegardée 

In [ ]:
df = df.sample(100)
df_traitee = pd.read_csv('data/données_calculees.csv')

### Ajout de la variable arrondissement

In [ ]:
df['Arrondissement'] = df.apply(lambda row: int(str(int(row.CodePostal))[3:5]), axis = 1)

# Couplage avec la base des données GPS

On souhaite obtenir les coordonnées GPS de tous les appartements présents dans la base.
Pour cela, nous avons trouvé une base de données qui répertorient toutes les adresses parisiennes et leur associe des coordonnées GPS.
Pour coupler les deux bases, nous utiliserons la variable id.
Ce code id est de la forme WWXXX_YYYY_ZZZZZ avec : 	
##### WW est le code département (75 ici)
##### XXX est le code commune (115 pour le 15ème arrondissement)
##### YYYY est le code voie (4903 pour l'avenue Jean Jaurès par exemple)
##### ZZZZ est le numéro (00005 pour le 5)


In [ ]:
df2 = pd.read_csv('https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-75.csv.gz', sep = ';').copy()

In [ ]:
df2[df2['id'].str[:2] == '75'].sample(5)

### Séléction des variables intéressantes

In [ ]:
df_GPS = df2[[
        'id', 'lon', 'lat', 
        ]]

# x et y : Coordonnées cartographique en projection légale
# lon et lat : Coordonnées en WGS-84	

df_GPS.sample(10)

### Couplage avec df

In [ ]:
print(df.shape)
df = df.merge(df_GPS, left_on = 'id', right_on = 'id')
print(df.shape)
# On perd a peine 1000 valeurs
display(df.sample(10))

On peut donc maintenant représenter les appartements dans un plan de Paris.

On constate bien les faits stylisé déjà connu : 
#### Les quartiers périphériques sont en général moins chers que les quartiers centraux.
#### Les quartiers de l'ouest sont plus chers que ceux à l'est.

# II. Traitement des données - ajout de variables


In [ ]:
df_ml = df.drop(columns = ['Date', 'Valeur', 'Surface',
                           'Voie', 'CodePostal', 'TypeLocal', 'id'])
df_ml.sample(10)

In [ ]:
corr_matrix = df_ml.corr()
corr_matrix["prixm2"].sort_values(ascending = False)
# les variables sont toutes plutôt décorélées avec le prix au m2

Pour les variables suivantes, on considère une petite proportion de la base totale (pour l'exemple, calculs trop longs)

## Quartier administatif

In [ ]:
df_quartier = pd.read_csv('data/quartier_paris.csv', sep = ';')

In [ ]:
df_quartier = df_quartier[['c_qu', 'geom']]
df_quartier['geom'] = df_quartier.apply(lambda row: json.loads(row.geom), axis = 1)
geom = [shape(i) for i in df_quartier['geom']]
df_quartier['geom'] = gpd.GeoDataFrame({'geometry':geom})['geometry']

In [ ]:
def determine_quartier(lat, lon):
    
    for _, row in df_quartier.iterrows():
        
        polygon = row.geom
        
        if polygon.contains(Point(lon, lat)):
            return row.c_qu
        
    return 0

In [ ]:
df_ml['Quartier'] = df_ml.apply(lambda row: determine_quartier(row.lat, row.lon), axis = 1)

## Proximité aux jardins 



In [ ]:
df_jardin = pd.read_csv('data/espaces_verts.csv', sep = ';')

In [ ]:
#On ne garde que les espaces verts importants
df_jardin = df_jardin[df_jardin['Catégorie'].isin(['Square', 'Jardin', 'Pelouse', 'Parc', 'Bois'])]

In [ ]:
df_jardin = df_jardin[['Superficie totale réelle', 'Geo Shape', '''Nom de l'espace vert''']].rename(columns = {
                                                                    'Superficie totale réelle' : 'Aire',
                                                                    'Geo Shape' : 'geom',
    '''Nom de l'espace vert''' : 'Nom'
                                                                                    }
                                                                       )
df_jardin = df_jardin.reset_index()

In [ ]:
df_jardin['geom'] = df_jardin.apply(lambda row: json.loads(str(row.geom)), axis = 1)
geom = [shape(i) for i in df_jardin['geom']]
df_jardin['geom'] = gpd.GeoDataFrame({'geometry':geom})['geometry']

In [ ]:
df_jardin = df_jardin.dropna(subset=['Aire'])

In [ ]:
pt = preprocessing.PowerTransformer(method='box-cox', standardize = False)
df_jardin[['Aire']] = pt.fit_transform(df_jardin[['Aire']])

min_max_scaler = preprocessing.MinMaxScaler()
df_jardin[['Aire']] = min_max_scaler.fit_transform(df_jardin[['Aire']])

In [ ]:
def calcule_score_jardin(lat, lon):
    
    L = []
    for _, row in df_jardin.iterrows():

        polygon = row.geom

        distance = polygon.distance(Point(lon, lat))
        L.append((distance, row.Aire))
        
    L = sorted(L, key=lambda x:  x[0])
    (distance, aire) = L[0]
    score = np.sqrt(aire)/ (distance * 111)
    
    return score

In [ ]:
df_ml['score_jardin'] = df_ml.apply(lambda row: calcule_score_jardin(row.lat, row.lon), axis = 1)

## Proximité aux monuments 


In [ ]:
df_monuments = pd.read_csv('data/monuments.csv', sep = ';')

# On standardise le nombre de visiteurs
df_monuments['visiteurs']/=max(df_monuments['visiteurs'])


In [ ]:
df_monuments.head()

### Calcul du score de proximité aux monuments

In [ ]:
def calcule_scores_monument(lat,lon):

    distances_aux_monuments = []

    for i,row in df_monuments.iterrows():
        distances_aux_monuments.append((row.nom, row.visiteurs, vincenty((row.lat, row.lon), (lat, lon))))

    try:
        score_1 = max([monument[1] for monument in distances_aux_monuments if monument[2] < 1])
    except:
        score_1 = 0

    try:
        score_2 = len([monument[1]/monument[2] for monument in distances_aux_monuments if monument[2] < 3])
    except:
        score_2 = 0

    return (score_1, score_2)

In [ ]:
L1, L2 = [], []

for _, row in df_ml.iterrows():
    (a,b) = calcule_scores_monument(row.lat, row.lon)
    L1.append(a)
    L2.append(b)

In [ ]:
df_ml['score_1'], df_ml['score_2'] = L1, L2

df_ml['score_2'] /= max(df_ml['score_2'])

df_ml['score_monument'] = ( (1/5) * df_ml['score_1'] + (4/5) * df_ml['score_2'] ) 

df_ml = df_ml.drop(columns = ['score_1', 'score_2'])
df_ml.head()

## Proximité aux stations de métro

On va ajouter une première variable : le rang des k (k=3 surement) stations de métro les plus proches d'un appartement.

On a besoin :

-d'une fonction qui calcule les distance entre chaque appartement et chaque station de métro. 

--> complexité(nb d'appart) =  nb de stations * nb d'appartement

-d'une fonction qui pour un appartement fixé, trouve les k distances les plus courtes 

--> complexité = k * nb de métro 

-de relier les stations avec leur rang

### Traitement de la base des stations parisiennes

In [ ]:
#Importation de la base

df_metro = pd.read_csv('data/stations_metro.csv', sep = ';')

df_metro.sample(3)

In [ ]:
df_metro.columns

In [ ]:
#On garde les colonnes et les lignes qui nous intéressent

df_metro = df_metro[[
        'Geo Point', 'nomlong', 'reseau',
                    ]]

df_metro = df_metro.where(df_metro['reseau'] == 'METRO').dropna()
df_metro.sample(3)

Problème, les coordonées géographiques doivent être traitées pour qu'elles soient exploitables

In [ ]:
#Fonction de standardisation des coordonnées 

def sepvirgulex(s):
    """
    """
    i = 0
    while s[i] != ',':
        i+=1
        
    x = float(s[:i])
    return x

def sepvirguley(s):
    """
    """
    i = 0
    while s[i] != ',':
        i+=1
        
    y = float(s[i+1:])
    
    return y

In [ ]:
# Crétaion des variables de lattitude et longitude à partir de la variables des coordonnées

df_metro['lat'] = df_metro['Geo Point'].apply(sepvirgulex)
df_metro['lon'] = df_metro['Geo Point'].apply(sepvirguley)

df_metro = df_metro[['nomlong','reseau','lon','lat']].rename(columns = {
                                                            'nomlong': 'Station', 
                                                                         }
                                                            )

df_metro.sample(5) #Comme x et y sont des floats, ils ne sont pas affichés complétement

### Base donnant la fréquentation des stations de métro

In [ ]:
#Importation de la base 

df_metro_pop = pd.read_csv('data/trafic_metro.csv', sep = ";")

In [ ]:
# On retient les colonnes et les lignes qui nous intéressent

df_metro_pop = df_metro_pop[['Rang','Station','Réseau','Trafic']].rename(columns = {'Réseau' : 'Reseau'})
df_metro_pop.where(df_metro_pop['Reseau'] == 'Métro').dropna()

df_metro_pop.sample(5)

In [ ]:
# Traitement du nom des stations pour qu'ils collent entre les deux bases

def sansparenthese(s):
    """
    """
    for i in range(len(s)):
        if s[i] == '(':
            
            return s[:i]
        
    return s

def sanstiret(s):
    """
    """
    for i in range(len(s)):
        if s[i]=='-':
            s = s[:i] + ' ' + s[i+1:]
            
    return s

def sansespace(s):
    """
    enleve les espaces placés après le nom de la station
    """
    if s[-1] != ' ':
        
        return s
    else:
        
        return sansespace(s[:-1])

def sansrer(s):
    """
    """
    if s[-4:]== ' RER':
        return s[:-4]
    
    return s



df_metro['Station'] = df_metro['Station'].apply(sanstiret)
df_metro['Station'] = df_metro['Station'].apply(sansparenthese)
df_metro['Station'] = df_metro['Station'].apply(sansespace)
df_metro['Station'] = df_metro['Station'].apply(sansrer)

df_metro = df_metro.replace([
    'LES COURTILLES ASNIERES GENNEVILLIERS', 'BOBIGNY PANTIN', 'SAINT MANDE', 'LOUVRE RIVOLI', 
    'MALAKOFF RUE ÉTIENNE DOLET', 'LA DEFENSE GRANDE ARCHE', 'BIBLIOTHEQUE FRANCOIS MITTERRAND', 
    'LES AGNETTES ASNIERES GENNEVILLIERS', 'PALAIS ROYAL MUSEE DU LOUVRE', "CHAUSSEE D'ANTIN", 
    'AUBERVILLIERS PANTIN', 'PLACE DE CLICHY', 'PEREIRE LEVALLOIS', 'JAVEL', 'MONTPARNASSE', 
    'GABRIEL PERI ASNIERES GENNEVILLIERS', 'FRANKLIN D.ROOSEVELT', 'AUSTERLITZ'
                            ], 

                           [
    'LES COURTILLES', 'BOBIGNY PANTIN RAYMOND QUENEAU', 'SAINT MANDE TOURELLE', 'LOUVRE', 
    'MALAKOFF RUE ETIENNE DOLET', 'LA DEFENSE', 'BIBLIOTHEQUE',
    'LES AGNETTES', 'PALAIS ROYAL', "CHAUSSEE D'ANTIN LA FAYETTE", 'AUBERVILLIERS PANTIN QUATRE CHEMINS',
    'PLACE CLICHY', 'PEREIRE', 'JAVEL ANDRE CITROEN', 'MONTPARNASSE BIENVENUE', 'GABRIEL PERI',
    'FRANKLIN D. ROOSEVELT', "GARE D'AUSTERLITZ"
                            ])

df_metro_pop = df_metro_pop.replace(['Métro'],['Metro'])
df_metro_pop['Station'] = df_metro_pop['Station'].apply(sanstiret)
df_metro_pop['Station'] = df_metro_pop['Station'].apply(sansparenthese)
df_metro_pop['Station'] = df_metro_pop['Station'].apply(sansespace)
df_metro_pop['Station'] = df_metro_pop['Station'].apply(sansrer)

In [ ]:
# On standardise la féréquentation

df_metro_pop['Trafic'] /= max(df_metro_pop['Trafic'])

On peut maintenant calculer le score

In [ ]:
def calcule_scores_station(lon,lat):

    distances_aux_stations = []

    for i,row in df_metro.iterrows():
        nom = df_metro.loc[i]['Station']
        popularite = max(df_metro_pop['Trafic'].where(df_metro_pop['Station'] == nom).dropna())
        distances_aux_stations.append((row.Station, popularite, vincenty((row.lat, row.lon), (lat, lon))))

    try:
        score_1 = max([station[1] for station in distances_aux_stations if station[2] < 0.2])
    except:
        score_1 = 0

    try:
        score_2 = max([station[1] for station in distances_aux_stations if station[2] < 0.7])
    except:
        score_2 = 0

    return score_1, score_2

In [ ]:
L1, L2 = [], []

for i, row in df_ml.iterrows():
    (a,b) = calcule_scores_station(row.lon, row.lat)
    L1.append(a)
    L2.append(b)

In [ ]:
df_ml['score_1'], df_ml['score_2'] = L1, L2

df_ml['score_metro'] = ( (1/5) * df_ml['score_1'] + (4/5) * df_ml['score_2'] ) 

df_ml = df_ml.drop(columns = ['score_1', 'score_2'])
df_ml.head()

# Proximité aux commerces

In [ ]:
#Importation des bases

#Source : Apur, Ville de Paris (DAE) et CCIP
#Infos supplémentaires sur les données
#https://geocatalogue.apur.org/catalogue/srv/fre/catalog.search#/metadata/970a0714-f9a6-48f7-8c89-ee8b8924876d
#https://opendata.apur.org/datasets/e769dbb901664c90bdbd05d73b94d7ee_0

api_url = "https://api-adresse.data.gouv.fr/search/?q=" # Lien qui nous donne l'adresse et les coordonnées GPS

df_commerce = pd.read_csv('data/commerces.csv', sep = ',')

In [ ]:
# On garde les lignes et colonnes qui nous intéressent

df_commerce = df_commerce[['ARRO', 'NUM', 'TYP_VOIE', 'LIB_VOIE', 'LIBACT' ]]
df_commerce.dropna()
df_commerce.sample(5)

On crée une fonction qui, à partir de la base de données, renvoie l'adresse selon les normes françaises, on utilise après cette pour retrouver les coordonnées

In [ ]:
def adri(i):
    if len(str(df_commerce.loc[i]['ARRO']))==2:
        adr = str(df_commerce.loc[i]['NUM']) + ', ' + str(df_commerce.loc[0]['TYP_VOIE']) + ' '+ str(df_commerce.loc[i]['LIB_VOIE']) + ', 750' + str(df_commerce.loc[i]['ARRO']) + ' Paris'
    else:
        adr = str(df_commerce.loc[i]['NUM']) + ', ' + str(df_commerce.loc[0]['TYP_VOIE']) + ' '+ str(df_commerce.loc[i]['LIB_VOIE']) + ', 7500' + str(df_commerce.loc[i]['ARRO']) + ' Paris'
    return adr

A chaque commerce on associe ses coorodonnées gps.

In [ ]:
df_commerce = pd.read_csv('data/commerces_gps.csv', sep = ',')

In [ ]:
pop_par_ar = [16252,20260,34788,27487,59108,40916,52512,36453,59269,91932,147017,141494,181552,137105,233484,165446,167835,195060,186393,195604]


df_com_arro = df_commerce['ARRO'].value_counts()
X = [str(i) for i in range(1,21)]
Y = [df_com_arro[i+1]/pop_par_ar[i] for i in range(20)]
plt.bar(X,Y)
plt.xlabel('Arrondissement')
plt.title('Nombre de commerces par habitant selon les arrondissements')
plt.show()

"""On remarque que des arrondissements comme le 1er ou le 4ème ont beaucoup de commerces par habitant, au contraire du 13ème qui lui possède  une part de comemrces par habitant plus faible"""

#source INSEE, qu'on peut aussi lire ici https://www.apur.org/sites/default/files/documents/recueil_thematique_1234_arr_paris_0.pdf
#on se donne les surfaces des arrondissements en hectare

surf_arrond =[182,99,117,160,254,215,409,388,218,289,367,639,715,562,850,790,567,600,679,598]
sum(surf_arrond)

Xs = [str(i) for i in range(1,21)]
Ys= [df_com_arro[i+1]/surf_arrond[i] for i in range(20)]

plt.bar(Xs,Ys)
plt.xlabel('Arrondissement')
plt.title('Nombre de commerces par hectare selon les arrondissements')
plt.show()

In [ ]:
"""Créeons une carte de densité des commerces"""

N = 29

maxlon = max(df_commerce['lon'])
minlon = min(df_commerce['lon'])
maxlat = 48.900391
minlat = min(df_commerce['lat'])

linsplon = list(np.linspace(minlon*(1 - 10**-7),maxlon*(1 +10**-7),N))
linsplat = list(np.linspace(minlat*(1 -10**-7),maxlat*(1 +10**-7),N))

def premier_plus_grand(L,x): #renvoie l'indice de la première occurence de la liste triée qui est strmt plus grande que x
    i=0
    while L[i]<= x:
        i+=1
    return i

def donne_zone(lat,lon,Linsplat,Linsplon):
  
    i = premier_plus_grand(Linsplat,lat)
    j = premier_plus_grand(Linsplon,lon)
    return i,j

zones_tout_commerce = np.zeros((N,N))
zones_commerce_luxe = np.zeros((N,N))

for k in range(len(df_commerce['lat'])):
    i,j = donne_zone(df_commerce['lat'][k],df_commerce['lon'][k], linsplat, linsplon)
    zones_tout_commerce[i][j] += 1
    
df_commerce_luxe = df_commerce[df_commerce['LIBACT'].isin(['Commerce détail de boissons', 
    'Charcuterie - Traiteur - Epicerie fine', 'Chocolaterie - Confiserie', 
    'Produits alimentaires bio et circuits courts', 'Crèmerie - Fromagerie', 
    'Torréfacteur - Commerce détail thé et café', 'Poisonnerie', 
    'Glacier : vente à emporter et consommation sur place', 'Alimentation générale de luxe > 300m²'
                                                            ]
                                                         )
                              ].reset_index(drop = True)
    
for k in range(len(df_commerce_luxe['lat'])):
    i,j = donne_zone(df_commerce_luxe['lat'][k], df_commerce_luxe['lon'][k], linsplat, linsplon)
    zones_commerce_luxe[i][j] += 1

plt.imshow(zones_commerce_luxe, cmap = 'Blues')

plt.colorbar()
plt.show()

def Zone_appart(lat,lon):
    i,j = donne_zone(lat,lon, linsplat, linsplon)
    return zones[i][j]

In [ ]:
def calcule_scores_commerces(lat,lon):
    
    i,j = donne_zone(lat, lon, linsplat, linsplon)
    score_1, score_2 = zones_tout_commerce[i][j], zones_commerce_luxe[i][j]
    
    return score_1, score_2

In [ ]:
L1, L2 = [], []

for i, row in df_ml.iterrows():
    #a = calcule_scores_commerces(row.lat, row.lon)
    (a,b) = calcule_scores_commerces(row.lat, row.lon)
    L1.append(a)
    L2.append(b)

In [ ]:
df_ml['score_commerce'], df_ml['score_commerce_lux'] = L1, L2 

df_ml['score_commerce'] /= max(df_ml['score_commerce'])
df_ml['score_commerce_lux'] /= max(df_ml['score_commerce_lux'])

# Data processing

In [174]:
df0 = pd.read_csv('data/données_calculees0.csv', index_col = 0)
df = df0.copy()

In [175]:
from sklearn import preprocessing

In [176]:
le = preprocessing.LabelEncoder()
df['TypeVoie'] = le.fit_transform(df['TypeVoie'].astype(str))

In [177]:
min_max_scaler = preprocessing.MinMaxScaler()
df[[
    'Numero', 'TypeVoie', 'NbPieces', 'lon', 'lat', 
    'periode_construction', 'Arrondissement', 'Quartier'
    ]] = min_max_scaler.fit_transform(df[[
                        'Numero', 'TypeVoie', 'NbPieces', 'lon', 'lat', 
                        'periode_construction', 'Arrondissement', 'Quartier'
                                        ]])

In [178]:
df = df[df['score_jardin'] < 8000]
df0[df0['score_jardin'] < 8000]

df[[
    'prixm2', 'score_commerce', 'score_commerce_lux','score_monument', 'score_jardin', 'score_metro'
    ]] = preprocessing.scale(df[[
            'prixm2', 'score_commerce', 'score_commerce_lux', 'score_monument', 'score_jardin', 'score_metro'
                                        ]])

In [179]:
#ax = pd.plotting.scatter_matrix(df[[
 #           'Numero', 'TypeVoie', 'NbPieces', 'lon', 'lat', 
  #  'periode_construction', 'Arrondissement', 'Quartier'
   #                                     ]].sample(1000), figsize = (12,12))
#ax
#plt.show()

# ML

In [180]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [181]:
y = df[['prixm2']]
X = df.drop(columns = 'prixm2')
lin_reg = LinearRegression().fit(X, y)
[(X.columns[i], lin_reg.coef_[0][i]) for i in range(13)]

[('Numero', -0.20707423199377809),
 ('TypeVoie', -0.10778615905631077),
 ('NbPieces', -0.133414609232699),
 ('lon', -0.10976400638069425),
 ('lat', -0.12157872170053065),
 ('score_commerce', -0.10309434834431527),
 ('score_commerce_lux', 0.10730259641203772),
 ('periode_construction', 0.06933286036092785),
 ('Arrondissement', 0.5451816190980654),
 ('Quartier', -0.9806005452516687),
 ('score_jardin', 0.005885863655249307),
 ('score_monument', 0.22171839751377773),
 ('score_metro', -0.03703966313098049)]

In [182]:
reg = Ridge(alpha=.5)
ridge_reg = reg.fit(X, y)
[(X.columns[i], ridge_reg.coef_[0][i]) for i in range(13)]

[('Numero', -0.2064882608507719),
 ('TypeVoie', -0.10770286865198112),
 ('NbPieces', -0.13290719442111742),
 ('lon', -0.11103833458445304),
 ('lat', -0.12156377406279963),
 ('score_commerce', -0.1031455374852953),
 ('score_commerce_lux', 0.10735351210439867),
 ('periode_construction', 0.06814520809953532),
 ('Arrondissement', 0.4214643985139861),
 ('Quartier', -0.8515933210016181),
 ('score_jardin', 0.005868901794017971),
 ('score_monument', 0.22155372351341338),
 ('score_metro', -0.03699103745775033)]

## ACP et Clustering

In [261]:
#df_test = df.drop(columns = ['lat', 'lon', 'Arrondissement', 'Quartier'])

In [262]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(df)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)


[0.30570026 0.21072141 0.15676292 0.15096819]
[220.35964146 182.95255675 157.79957268 154.85558923]


In [263]:
transformed_df = pd.DataFrame(pca.transform(df)).rename(columns = {
                            0 : 'var0', 
                            1 : 'var1', 
                            2 : 'var2',
                            3 : 'var3'
                         })

In [264]:
px.scatter_3d(transformed_df.head(1000), x = 'var0', y = 'var1', z = 'var2', color = 'var3', opacity = 1)

In [265]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 4).fit(transformed_df)

In [266]:
transformed_df['cluster'] = kmeans.labels_

In [267]:
px.scatter_3d(transformed_df.head(1000), x = 'var0', y = 'var1', z = 'var2', color = 'cluster', opacity = .7)

In [268]:
df_cluster0 = df0.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 0].index)]]
df_cluster1 = df0.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 1].index)]]
df_cluster2 = df0.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 2].index)]]
df_cluster3 = df0.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 3].index)]]

In [269]:
display(df_cluster0.describe())
display(df_cluster1.describe())
display(df_cluster2.describe())
display(df_cluster3.describe())

,Numero,NbPieces,prixm2,lon,lat,score_commerce,score_commerce_lux,periode_construction,Arrondissement,Quartier,score_jardin,score_monument,score_metro,cluster
count,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6686.000000,6684.000000
mean,38.293898,2.566856,14.609217,2.326728,48.862388,0.142096,0.145049,3.242746,10.278941,39.669309,6.202669,0.511888,0.145901,0.258677
std,45.428461,1.506586,5.314642,0.029321,0.014826,0.079198,0.100632,1.991118,5.431904,21.626267,12.241367,0.181414,0.079248,0.585851
min,1.000000,0.000000,4.950000,2.257939,48.822362,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,1.000000,11.278333,2.299889,48.851583,0.075472,0.066667,2.000000,6.000000,21.000000,2.345138,0.400000,0.088275,0.000000
50%,22.000000,2.000000,12.927891,2.333067,48.862717,0.141509,0.133333,3.000000,9.000000,36.000000,3.572149,0.504412,0.128687,0.000000
75%,48.000000,3.000000,16.000000,2.348380,48.874038,0.198113,0.200000,4.000000,16.000000,62.000000,6.085205,0.604412,0.166921,0.000000
max,400.000000,11.000000,35.182926,2.408703,48.897526,0.481132,0.566667,11.000000,20.000000,80.000000,661.925835,0.950000,0.815167,3.000000


,Numero,NbPieces,prixm2,lon,lat,score_commerce,score_commerce_lux,periode_construction,Arrondissement,Quartier,score_jardin,score_monument,score_metro,cluster
count,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000,12719.000000
mean,51.312525,2.326755,9.326415,2.341597,48.862165,0.122799,0.072105,3.518044,15.934901,62.131142,8.737383,0.173153,0.103369,1.022329
std,58.704226,1.170078,1.954469,0.044089,0.023219,0.080507,0.077799,2.082313,3.137316,12.560147,27.123597,0.123727,0.054114,0.383827
min,1.000000,0.000000,4.230769,2.255662,48.819437,0.000000,0.000000,0.000000,2.000000,7.000000,0.736999,0.000000,0.000000,0.000000
25%,11.000000,1.000000,8.111257,2.303850,48.840159,0.056604,0.000000,3.000000,14.000000,55.000000,3.011984,0.050000,0.069560,1.000000
50%,29.000000,2.000000,9.315789,2.346721,48.863732,0.103774,0.033333,3.000000,16.000000,62.000000,4.621010,0.150000,0.089263,1.000000
75%,70.000000,3.000000,10.501470,2.380464,48.885897,0.179245,0.100000,4.000000,18.000000,71.000000,7.995657,0.250000,0.130620,1.000000
max,407.000000,20.000000,19.239473,2.412651,48.900391,0.424528,0.366667,11.000000,20.000000,80.000000,2053.118551,0.667768,0.386269,3.000000


,Numero,NbPieces,prixm2,lon,lat,score_commerce,score_commerce_lux,periode_construction,Arrondissement,Quartier,score_jardin,score_monument,score_metro,cluster
count,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3929.000000,3928.000000
mean,44.801476,2.310257,10.819738,2.337973,48.873634,0.434241,0.444252,3.151693,13.576483,52.708323,5.574791,0.366694,0.141545,1.545570
std,50.563669,1.161018,3.088319,0.027282,0.016018,0.176713,0.168920,1.978473,5.506043,21.871091,8.401103,0.178288,0.084649,0.684107
min,1.000000,0.000000,4.234375,2.272877,48.833449,0.188679,0.100000,0.000000,1.000000,2.000000,1.074346,0.050000,0.034916,0.000000
25%,10.000000,1.000000,9.055555,2.318332,48.863447,0.311321,0.333333,2.000000,10.000000,38.000000,2.320529,0.204412,0.082528,1.000000
50%,28.000000,2.000000,10.535714,2.342618,48.878218,0.377358,0.400000,3.000000,17.000000,65.000000,3.369736,0.362922,0.120235,2.000000
75%,61.000000,3.000000,12.135922,2.356190,48.886545,0.481132,0.533333,3.000000,18.000000,70.000000,5.545529,0.502229,0.153319,2.000000
max,397.000000,9.000000,34.600000,2.392678,48.894568,1.000000,1.000000,11.000000,20.000000,77.000000,160.438533,0.850000,0.601794,3.000000


,Numero,NbPieces,prixm2,lon,lat,score_commerce,score_commerce_lux,periode_construction,Arrondissement,Quartier,score_jardin,score_monument,score_metro,cluster
count,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000,1634.000000
mean,48.222766,2.568543,11.070875,2.341756,48.867310,0.163697,0.140596,3.230722,10.123623,39.037332,5.824784,0.453436,0.691726,2.905141
std,55.804833,1.394773,3.330491,0.019601,0.016270,0.099123,0.129461,1.924162,2.973760,11.792727,7.819163,0.184820,0.160854,0.442877
min,1.000000,0.000000,4.238200,2.311758,48.835830,0.000000,0.000000,0.000000,1.000000,2.000000,1.002227,0.150000,0.251369,0.000000
25%,9.000000,1.000000,9.168750,2.323015,48.847725,0.103774,0.033333,2.000000,8.000000,32.000000,2.379624,0.304412,0.583827,3.000000
50%,23.000000,2.000000,10.618143,2.347738,48.875687,0.150943,0.133333,3.000000,10.000000,37.000000,3.456994,0.450000,0.740528,3.000000
75%,69.000000,3.000000,12.352206,2.358749,48.880082,0.207547,0.200000,3.000000,10.750000,42.250000,5.945085,0.617703,0.806881,3.000000
max,245.000000,11.000000,29.894736,2.383882,48.886033,0.490566,0.666667,11.000000,18.000000,72.000000,87.752273,0.850000,1.000000,3.000000


In [270]:
df0['cluster'] = pd.Series(kmeans.labels_)

In [283]:
px.scatter(df0, x = 'lon', y = 'lat', color = 'score_commerce_lux', template = 'none', opacity = .3)